<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Forum-Activity-Report" data-toc-modified-id="Forum-Activity-Report-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Forum Activity Report</a></span></li><li><span><a href="#Specify-Report-Duration" data-toc-modified-id="Specify-Report-Duration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Specify Report Duration</a></span></li><li><span><a href="#Get-Forum-Activity-Data" data-toc-modified-id="Get-Forum-Activity-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get Forum Activity Data</a></span></li><li><span><a href="#Capture-Markdown-Report" data-toc-modified-id="Capture-Markdown-Report-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Capture Markdown Report</a></span></li><li><span><a href="#Place-Activity-Report-in-Markdown-Cell" data-toc-modified-id="Place-Activity-Report-in-Markdown-Cell-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Place Activity Report in Markdown Cell</a></span></li></ul></div>

# Forum Activity Report
The notebook generates a report of posts in specific categories for the duration specified.

Enter the begin and end dates in the top cell, execute all the remaining cells, and paste the output of the last cell in a markdown cell (or in the Standup google doc) to see the output. 

Caution: There are still some bugs or unhandled cases. So use the output and validate against the Latest page in the discussion forum. Modify or remove entries as appropriate.

# Specify Report Duration
Set the from_date and to_date in the cell below. The MAX_PAST_PAGES specifies maximum paginations in the recent posts (the Discourse API does not take an explicit date range).

In [31]:
### SET FROM AND TO DATE ##########
from_date = '2022-03-01'
to_date = '2022-04-01'
###################################
## INCREASE IT IF DURATION RANGES BEYOND THE RECENT 2 MONTHS
MAX_PAST_PAGES = 10

# Get Forum Activity Data
Obtain the data by making API requests from the forum server.

In [32]:
import requests
import json
import pprint
from requests.exceptions import HTTPError

print('Processing from_date: {}, to_date: {}'.format(from_date, to_date))

URL_PREFIX = 'https://discuss.aerospike.com/t/'
release_posts = []
kb_posts = []
announce_posts = []
question_posts = []

curr_page = 0
curr_page_min_date = None
while (curr_page < MAX_PAST_PAGES and (curr_page_min_date == None or curr_page_min_date >= from_date)):
    try:
        response = requests.get('https://discuss.aerospike.com/latest.json?page='+str(curr_page))

        response.raise_for_status()
        # access JSOn content
        jsonResponse = response.json()
        #parsed = json.loads(jsonResponse)
        #print("Entire JSON response")
        #pp = pprint.PrettyPrinter(indent=1)
        #pp.pprint(jsonResponse)
        #print(json.dumps(parsed))

    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    
    topics = jsonResponse['topic_list']['topics']
    
    # relevant fields and formats: 
    #      'bumped_at': '2022-01-29T12:58:04.851Z', 'category_id': 152, 'id': 9090
    #.     'slug': 'aerospike-rest-client-release-1-9-1-december-16-2021',
    #.     'last_poster_username': 'Aerospike_Knowledge'
    #      'title': 'How to add a new node to a cluster approaching utilisation limits (HWM or stop-writes)'

    for topic in topics:
        topic_id = topic['id']
        topic_date = topic['bumped_at'][:10]
        #print('topic date: ', topic_date)
        
        if topic_id != 4526: # reposted old announcement
            if curr_page_min_date == None or topic_date < curr_page_min_date:
                curr_page_min_date = topic_date

            if topic_date > to_date or topic_date < from_date:
                continue
            
        topic_info = { 'id': topic['id'], 'date': topic['bumped_at'][:10], 'title': topic['title'], 
                               'url': URL_PREFIX+topic['slug']+'/'+str(topic['id']) }
        #print("topic info: ", topic_info)

        #.    if category_id == 152 -> process release
        if topic['category_id'] == 152:
            release_posts.append(topic_info)
            continue

        #.    else if category_id == 3 -> process announcements (ignore)
        if topic['category_id'] == 3:
            announce_posts.append(topic_info)
            continue

        #.    else if last_poster_username == Aerospike_Knowledge -> process knowledgebase
        if topic['last_poster_username'] == 'Aerospike_Knowledge':
            kb_posts.append(topic_info)
            continue

        #.    else process questions (may need to check against negative conditions)
        question_posts.append(topic_info)
        
    print('curr_page: {}, curr-page-min-date: {}'.format(curr_page, curr_page_min_date))
    curr_page += 1
print('Done.')

Processing from_date: 2022-03-01, to_date: 2022-04-01
curr_page: 0, curr-page-min-date: 2022-03-11
curr_page: 1, curr-page-min-date: 2022-02-14
Done.


# Capture Markdown Report
The following cell captures the print output in the variable "activity_report" which is used later.

In [33]:
%%capture activity_report

print('# Forum Activity Report From {} To {}'.format(from_date, to_date))
print('\n## Questions from the Community')
#pp.pprint(question_posts)
for p in sorted(question_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))

print('\n## Knowledgebase Articles')
#pp.pprint(kb_posts)
for p in sorted(kb_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))

print('\n## Release Notes', end='')
#pp.pprint(release_posts)
from datetime import date
import re
# extract product and version info
prev_date = None
prev_prod = None
for p in sorted(release_posts, key=lambda x: x['date']+x['title']+str(x['id']), reverse=True):
    #print(p['title'])
    prod_ver = re.match('Aerospike\W*(\D+)([0-9.]\S*)', re.sub('version|Release', '', p['title']))
    if not prod_ver:
        continue
    pv_info = prod_ver.groups()
    prod = pv_info[0].rstrip()
    version = pv_info[1]
    if prev_date != p['date']:
        print('\n- {}'.format(date.fromisoformat(p['date']).strftime("%B %d")), end='')
    if prev_prod == prod:
        print(', [{}]({})'.format(version, p['url']), end='')
    else:   
        print('\n\t- {} [{}]({})'.format(prod, version, p['url']), end='')
    prev_date = p['date']
    prev_prod = prod
    
print('\n\n## Announcements')
#pp.pprint(announce_posts)
for p in sorted(announce_posts, key=lambda x: x['date']+str(x['id']), reverse=True):
    print('- [{}]({})'.format(p['title'], p['url']))


# Place Activity Report in Markdown Cell 
Place the "activity_report" above in a new cell. From the menu, change the type of the new cell to Markdown, and run the cell to view the report.

In [34]:
from IPython.display import display, Markdown
display(Markdown(activity_report.stdout))
get_ipython().set_next_input(activity_report.stdout)

# Forum Activity Report From 2022-03-01 To 2022-04-01

## Questions from the Community
- [Unable to launch GKE cluster](https://discuss.aerospike.com/t/unable-to-launch-gke-cluster/9056)
- [Java Client Error Message : Error -3,1,0,30000,0,2: Cluster is empty](https://discuss.aerospike.com/t/java-client-error-message-error-3-1-0-30000-0-2-cluster-is-empty/9289)
- [Multiple FilterExp query with aggregation](https://discuss.aerospike.com/t/multiple-filterexp-query-with-aggregation/9281)
- [Truncate set by mistake,how to quickly restore data?](https://discuss.aerospike.com/t/truncate-set-by-mistake-how-to-quickly-restore-data/9292)
- [EAGAIN (Resource temporarily unavailable) error during asrestore](https://discuss.aerospike.com/t/eagain-resource-temporarily-unavailable-error-during-asrestore/9273)
- [UDF problem with maps](https://discuss.aerospike.com/t/udf-problem-with-maps/9270)
- [Bin convergence and Support of multiple counters in XDR](https://discuss.aerospike.com/t/bin-convergence-and-support-of-multiple-counters-in-xdr/9267)
- [How to stream UDF result to client to reduce memory CPU consuption](https://discuss.aerospike.com/t/how-to-stream-udf-result-to-client-to-reduce-memory-cpu-consuption/9097)
- [Any way return csv or at lest plain text from the AQL?](https://discuss.aerospike.com/t/any-way-return-csv-or-at-lest-plain-text-from-the-aql/9083)
- [Unexpected stop-writes](https://discuss.aerospike.com/t/unexpected-stop-writes/9278)
- [Is there a way to reset the evit counter on the stats?](https://discuss.aerospike.com/t/is-there-a-way-to-reset-the-evit-counter-on-the-stats/9272)
- [Copy bins between records in place](https://discuss.aerospike.com/t/copy-bins-between-records-in-place/9276)
- [Data loss when used asbackup during migrations](https://discuss.aerospike.com/t/data-loss-when-used-asbackup-during-migrations/9274)
- [Aerospike tries to connect to dead node](https://discuss.aerospike.com/t/aerospike-tries-to-connect-to-dead-node/8609)
- [Expiration of records is late only on one node (out of six)](https://discuss.aerospike.com/t/expiration-of-records-is-late-only-on-one-node-out-of-six/9266)
- [Move data from mem backed namespace to disk backed namespace based on ttl](https://discuss.aerospike.com/t/move-data-from-mem-backed-namespace-to-disk-backed-namespace-based-on-ttl/9256)
- [Ranking using score and pagination support in Aerospike](https://discuss.aerospike.com/t/ranking-using-score-and-pagination-support-in-aerospike/9250)

## Knowledgebase Articles
- [Node Not Found For Partition error after a full restart of strongly consistent namespace](https://discuss.aerospike.com/t/node-not-found-for-partition-error-after-a-full-restart-of-strongly-consistent-namespace/9225)

## Release Notes
- April 01
	- Java Client [6.0.0](https://discuss.aerospike.com/t/aerospike-java-client-release-6-0-0-march-31-2022/9308)
	- C# Client [5.0.0](https://discuss.aerospike.com/t/aerospike-c-client-release-5-0-0-march-31-2022/9307)
	- C Client [6.0.0](https://discuss.aerospike.com/t/aerospike-c-client-release-6-0-0-march-31-2022/9306)
- March 31
	- Database [5.7.0.15](https://discuss.aerospike.com/t/aerospike-database-5-7-0-15-march-31-2022/9304), [5.5.0.25,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-25-5-6-0-19-march-30-2022/9303)
- March 22, [5.5.0.23,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-23-5-6-0-17-5-7-0-12-march-21-2022/9298)
- March 15
	- Node.js Client [4.0.0](https://discuss.aerospike.com/t/aerospike-node-js-client-release-4-0-0-march-14-2022/9285)
- March 14
	- Monitoring Stack [1.4.0](https://discuss.aerospike.com/t/aerospike-monitoring-stack-release-1-4-0-march-14-2022/9284)
- March 09
	- Database [5.5.0.22,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-22-5-6-0-16-5-7-0-11-march-8-2022/9269)
- March 03
	- Connect for Presto - Trino [3.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-3-1-0-march-3-2022/9254), [1.8.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-8-0-march-3-2022/9253)
	- C Client [5.2.8](https://discuss.aerospike.com/t/aerospike-c-client-release-5-2-8-march-3-2022/9252)

## Announcements
- [Cross posting to/from other sites (such as Stack Overflow)](https://discuss.aerospike.com/t/cross-posting-to-from-other-sites-such-as-stack-overflow/4526)


In [ ]:
# Forum Activity Report From 2022-03-01 To 2022-04-01

## Questions from the Community
- [Unable to launch GKE cluster](https://discuss.aerospike.com/t/unable-to-launch-gke-cluster/9056)
- [Java Client Error Message : Error -3,1,0,30000,0,2: Cluster is empty](https://discuss.aerospike.com/t/java-client-error-message-error-3-1-0-30000-0-2-cluster-is-empty/9289)
- [Multiple FilterExp query with aggregation](https://discuss.aerospike.com/t/multiple-filterexp-query-with-aggregation/9281)
- [Truncate set by mistake,how to quickly restore data?](https://discuss.aerospike.com/t/truncate-set-by-mistake-how-to-quickly-restore-data/9292)
- [EAGAIN (Resource temporarily unavailable) error during asrestore](https://discuss.aerospike.com/t/eagain-resource-temporarily-unavailable-error-during-asrestore/9273)
- [UDF problem with maps](https://discuss.aerospike.com/t/udf-problem-with-maps/9270)
- [Bin convergence and Support of multiple counters in XDR](https://discuss.aerospike.com/t/bin-convergence-and-support-of-multiple-counters-in-xdr/9267)
- [How to stream UDF result to client to reduce memory CPU consuption](https://discuss.aerospike.com/t/how-to-stream-udf-result-to-client-to-reduce-memory-cpu-consuption/9097)
- [Any way return csv or at lest plain text from the AQL?](https://discuss.aerospike.com/t/any-way-return-csv-or-at-lest-plain-text-from-the-aql/9083)
- [Unexpected stop-writes](https://discuss.aerospike.com/t/unexpected-stop-writes/9278)
- [Is there a way to reset the evit counter on the stats?](https://discuss.aerospike.com/t/is-there-a-way-to-reset-the-evit-counter-on-the-stats/9272)
- [Copy bins between records in place](https://discuss.aerospike.com/t/copy-bins-between-records-in-place/9276)
- [Data loss when used asbackup during migrations](https://discuss.aerospike.com/t/data-loss-when-used-asbackup-during-migrations/9274)
- [Aerospike tries to connect to dead node](https://discuss.aerospike.com/t/aerospike-tries-to-connect-to-dead-node/8609)
- [Expiration of records is late only on one node (out of six)](https://discuss.aerospike.com/t/expiration-of-records-is-late-only-on-one-node-out-of-six/9266)
- [Move data from mem backed namespace to disk backed namespace based on ttl](https://discuss.aerospike.com/t/move-data-from-mem-backed-namespace-to-disk-backed-namespace-based-on-ttl/9256)
- [Ranking using score and pagination support in Aerospike](https://discuss.aerospike.com/t/ranking-using-score-and-pagination-support-in-aerospike/9250)

## Knowledgebase Articles
- [Node Not Found For Partition error after a full restart of strongly consistent namespace](https://discuss.aerospike.com/t/node-not-found-for-partition-error-after-a-full-restart-of-strongly-consistent-namespace/9225)

## Release Notes
- April 01
	- Java Client [6.0.0](https://discuss.aerospike.com/t/aerospike-java-client-release-6-0-0-march-31-2022/9308)
	- C# Client [5.0.0](https://discuss.aerospike.com/t/aerospike-c-client-release-5-0-0-march-31-2022/9307)
	- C Client [6.0.0](https://discuss.aerospike.com/t/aerospike-c-client-release-6-0-0-march-31-2022/9306)
- March 31
	- Database [5.7.0.15](https://discuss.aerospike.com/t/aerospike-database-5-7-0-15-march-31-2022/9304), [5.5.0.25,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-25-5-6-0-19-march-30-2022/9303)
- March 22, [5.5.0.23,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-23-5-6-0-17-5-7-0-12-march-21-2022/9298)
- March 15
	- Node.js Client [4.0.0](https://discuss.aerospike.com/t/aerospike-node-js-client-release-4-0-0-march-14-2022/9285)
- March 14
	- Monitoring Stack [1.4.0](https://discuss.aerospike.com/t/aerospike-monitoring-stack-release-1-4-0-march-14-2022/9284)
- March 09
	- Database [5.5.0.22,](https://discuss.aerospike.com/t/aerospike-database-5-5-0-22-5-6-0-16-5-7-0-11-march-8-2022/9269)
- March 03
	- Connect for Presto - Trino [3.1.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-3-1-0-march-3-2022/9254), [1.8.0](https://discuss.aerospike.com/t/aerospike-connect-for-presto-trino-1-8-0-march-3-2022/9253)
	- C Client [5.2.8](https://discuss.aerospike.com/t/aerospike-c-client-release-5-2-8-march-3-2022/9252)

## Announcements
- [Cross posting to/from other sites (such as Stack Overflow)](https://discuss.aerospike.com/t/cross-posting-to-from-other-sites-such-as-stack-overflow/4526)
